<a href="https://colab.research.google.com/github/zerualem/pi-project/blob/master/Pi_data_using_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [6]:
!dir

My\ Drive


In [8]:
%cd My\ Drive/PIPM_Data_R_code/Analysis

/gdrive/My Drive/PIPM_Data_R_code/Analysis


In [9]:
import pandas as pd

all_dmy = pd.read_csv('all_piData_dmy.csv')
all_dmy.shape

(26258, 238)

In [20]:
all_dmy.columns[all_dmy.columns.str.startswith('pressure_injury_poa')]

Index(['pressure_injury_poa.Yes'], dtype='object')

In [10]:
y = all_dmy[['pi']]
X = all_dmy.drop(['pi', 'pi_dayfromadmit', 'pressure_injury_poa.Yes'], axis=1)
X.shape

(26258, 235)

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [23]:
print(X_train.shape)
print(y_train.shape)

(19693, 235)
(19693, 1)


In [24]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
print(scaler.fit(X_train))
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

MinMaxScaler(copy=True, feature_range=(0, 1))


In [15]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.14.0


In [25]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)
y_train.shape

(19693, 2)

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(X_train.shape[1],)),
    keras.layers.Dense(1000, activation=tf.nn.relu),
    keras.layers.Dropout(rate=0.4),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(2, activation=tf.nn.softmax)
])

See: https://www.tensorflow.org/beta/tutorials/keras/imbalanced_data

https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc

In [0]:
metrics = [
      keras.metrics.Accuracy(name='accuracy'),
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc')]

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=metrics)

In [34]:
EPOCHS = 10
BATCH_SIZE = 2048

history = model.fit(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS)

Epoch 1/10
19693/19693 [==============================] - 1s 68us/sample - loss: 3.1130 - accuracy: 0.7959 - tp: 15702.0000 - fp: 3991.0000 - tn: 15702.0000 - fn: 3991.0000 - precision: 0.7973 - recall: 0.7973 - auc: 0.7976
Epoch 2/10
19693/19693 [==============================] - 1s 61us/sample - loss: 0.7684 - accuracy: 0.9500 - tp: 18709.0000 - fp: 984.0000 - tn: 18709.0000 - fn: 984.0000 - precision: 0.9500 - recall: 0.9500 - auc: 0.9500
Epoch 3/10
19693/19693 [==============================] - 1s 60us/sample - loss: 0.7684 - accuracy: 0.9500 - tp: 18709.0000 - fp: 984.0000 - tn: 18709.0000 - fn: 984.0000 - precision: 0.9500 - recall: 0.9500 - auc: 0.9500
Epoch 4/10
19693/19693 [==============================] - 1s 59us/sample - loss: 0.7684 - accuracy: 0.9500 - tp: 18709.0000 - fp: 984.0000 - tn: 18709.0000 - fn: 984.0000 - precision: 0.9500 - recall: 0.9500 - auc: 0.9500
Epoch 5/10
19693/19693 [==============================] - 1s 59us/sample - loss: 0.7684 - accuracy: 0.9500 - t

In [39]:
test_loss, test_acc, test_tp, test_fp, test_tn, test_fn, test_prec, test_rec, test_auc = model.evaluate(X_test, y_test)

print(f'Accuracy: {test_acc} , tp: {test_tp}, tfp: {test_fp}, tn: {test_tn}, fn: {test_fn}')

6565/6565 [==============================] - 1s 78us/sample - loss: 0.8105 - accuracy: 0.9473 - tp: 6219.0000 - fp: 346.0000 - tn: 6219.0000 - fn: 346.0000 - precision: 0.9473 - recall: 0.9473 - auc: 0.9473
Accuracy: 0.9472962617874146 , tp: 6219.0, tfp: 346.0, tn: 6219.0, fn: 346.0


In [40]:
from sklearn import metrics
predicted = model.predict(X_test)
print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6219
           1       0.00      0.00      0.00       346

   micro avg       0.95      0.95      0.95      6565
   macro avg       0.47      0.50      0.49      6565
weighted avg       0.90      0.95      0.92      6565
 samples avg       0.95      0.95      0.95      6565



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
